In [12]:
from pyfaidx import Fasta

def set_org_name_dict(codenames1, codenames2):
    codefiles=[codenames1, codenames2]
    org_dict={}
    for c in codefiles:
        orgs=open(c).readlines()
        for o in orgs:
            vec=o.split("\t")
            organism=vec[0].split("/")[-1].replace("fasta","").replace("faa","").split("_")[0]
            code=vec[1].replace("\n","")
            org_dict[code]=organism
    return org_dict

#get original organism name from a orthomcl protein ID
def pull_org_id(prot, org_dict):
    org_id=prot.split("|")[0]
    orig_name=org_dict.get(org_id,"NA")
    return orig_name

In [13]:
def get_prot_lens(faa_file):
    len_dict={}
    f=Fasta(faa_file)
    for i in f.keys():
        name=i
        length=len(str(f[i]))
        len_dict[name]=length
    return len_dict


def filter_blast(input1, faa, outfile, name_dict, cov_thresh=75):
    len_dict=get_prot_lens(faa)
    out=open(outfile, "w")
    out.write("name\tphage\tlocus_tag\thitt\thost\thost_locus_tag\taln_len\tpct_id\teval\tpct_cov\n")
    with open(input1) as infile:
        for line in infile:
            name=line.split("\t")[0]
            phage=pull_org_id(name, name_dict)
            lt=name.split("|")[1]
            
            hit=line.split("\t")[1]
            host=pull_org_id(hit.split("|")[0], name_dict)
            host_lt=hit.split("|")[1]
            
            prot_len=len_dict[name]
            aln_len=int(line.split("\t")[3])
            pct_id=float(line.split("\t")[2])
            ev=line.split("\t")[-2]
            pct_cov=(aln_len/prot_len)*100
            
            if pct_id>35 and pct_cov>cov_thresh:
                out.write(("\t".join([name, phage, lt, hit, host,host_lt, str(aln_len), str(pct_id), str(ev), str(pct_cov)]))+"\n")

    out.close()


In [14]:
name_dict=set_org_name_dict("../OrthoMCL/phage_v_phage/phage_taxon_codes.txt","../OrthoMCL/host_taxon_codes.txt")

filter_blast("phage_vs_host.out","allphage_goodProteins.fasta", "pvh_blast_summary.txt",name_dict)

In [15]:
blast=open("pvh_blast_summary.txt").readlines()
out=open("pvh_blast_cytoscape1.txt","w")
out_ponly=open("pvh_blast_cytoscape2.txt","w")
out_ponlyi=open("pvh_blast_cytoscape2_info.txt","w")
out_ponly.write("organism\tprotein\n")
out_ponlyi.write("name\ttype\n")
out2=open("pvh_blast_cytoscape1_nodeinfo.txt","w")
out2.write("name\ttype\n")
out.write("organism\tprotien\n")
for l in blast[1:]:
    phage=l.split("\t")[1]
    host=l.split("\t")[4]
    prot=l.split("\t")[5]
    out.write("%s\t%s\n%s\t%s\n"% (phage, prot, host, prot))
    out_ponly.write("%s\t%s\n"% (phage, prot))
    out_ponlyi.write("%s\tphage\n%s\tprotein\n"% (phage, prot))
    out2.write("%s\tphage\n%s\thost\n%s\tprotein\n"% (phage, host, prot))
    
out.close()
out2.close()
out_ponly.close()
out_ponlyi.close()
!open pvh_blast_cytoscape* -a textedit


In [3]:
name_dict=set_org_name_dict("../OrthoMCL/phage_v_phage/phage_taxon_codes.txt","../OrthoMCL/host_taxon_codes.txt")

filter_blast("phage_vs_host_derep.out","allphage_goodProteins.fasta", "pvh_derep_blast_summary.txt",name_dict)

In [5]:
blast=open("pvh_derep_blast_summary.txt").readlines()
out=open("pvh_derep_blast_cytoscape1.txt","w")
out_ponly=open("pvh_derep_blast_cytoscape2.txt","w")
out_ponlyi=open("pvh_derep_blast_cytoscape2_info.txt","w")
out_ponly.write("organism\tprotein\n")
out_ponlyi.write("name\ttype\n")
out2=open("pvh_blast_cytoscape1_nodeinfo.txt","w")
out2.write("name\ttype\n")
out.write("organism\tprotien\n")
for l in blast[1:]:
    phage=l.split("\t")[1]
    host=l.split("\t")[4]
    prot=l.split("\t")[5]
    out.write("%s\t%s\n%s\t%s\n"% (phage, prot, host, prot))
    out_ponly.write("%s\t%s\n"% (phage, prot))
    out_ponlyi.write("%s\tphage\n%s\tprotein\n"% (phage, prot))
    out2.write("%s\tphage\n%s\thost\n%s\tprotein\n"% (phage, host, prot))
    
out.close()
out2.close()
out_ponly.close()
out_ponlyi.close()
!open pvh_derep_blast_cytoscape* -a textedit


In [17]:
f=open("pvh_derep_blast_summary.txt").readlines()
protlist=[]

for l in f:
    vec=l.split("\t")
    hit=vec[5]
    if hit not in protlist:
        protlist.append(hit)